In [195]:
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.nn.functional import one_hot
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#### Dataset

Download dataset

In [196]:
transform = transforms.Compose([
    transforms.ToTensor()
    ])

In [197]:
train_set = CIFAR10(root='./data', download=True, transform=transform)
test_set = CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


One-hot encode labels

In [198]:
# train_set.targets = one_hot(torch.tensor(train_set.targets), len(train_set.classes))
# test_set.targets = one_hot(torch.tensor(test_set.targets), len(train_set.classes))

Load training and test dataset

In [199]:
batch_size = 256
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

Check sizes 

In [200]:
for i, (x, y) in enumerate(train_loader):
    print(x.shape)
    print(y.shape)
    if i == 0:
        break

torch.Size([256, 3, 32, 32])
torch.Size([256])


#### Model

Define model

In [201]:
class VGGNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding="same")
        self.conv2 = nn.Conv2d(32, 32, 3, padding="same")
        self.max_pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)


In [202]:
model = VGGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [203]:
epochs = 2

In [207]:
for e in range(epochs):
    training_loss = 0
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        if i == 0:
            break
training_loss

2.302245616912842